In [1]:
from IPython.display import clear_output
from data_generator import DataGenerator
from medium_generator import MediumGenerator, disk_func, cosine_func
import numpy as np
from ngsolve import * 
from ngsolve.webgui import Draw

dg = DataGenerator(maxh = (0.02, 0.04))

medium = MediumGenerator(cosine_func) # MediumGenerator(disk_func)

background_params = [{"type": 0, "x": 0.0, "y": 0.0, "r": 0.5, "v": 0.0}]
background_permittivity = medium.generate(background_params)



params  = [{"type": 0, "x": 0, "y": 0, "r": 0.5, "v": 1}]

permittivity  = medium.generate(params)


Mesh generation took 6.05076428106986 seconds


In [2]:
direction_angles = np.linspace(0, 2 * np.pi, 32, endpoint=False)

cauchy_data = dg.generate_cauchy_data(2 * pi, direction_angles, permittivity)

Solving took 5.089801823953167 seconds


KeyboardInterrupt: 

In [ ]:
direction_angles = np.linspace(0, 2 * np.pi, 4, endpoint=False)

A = Matrix(len(direction_angles) ** 2, dg.fes.ndof, complex=True)
b = Vector(len(direction_angles) ** 2, complex=True)

l = 0
for i in range(4):

    kx = 2 * pi * cos(direction_angles[i])
    ky = 2 * pi * sin(direction_angles[i])

    psi1 = CF((exp(1j * kx * x) * exp(1j * ky * y)))

    u_scat = dg.solve(kx, ky, permittivity)
    uB_scat0 = dg.solve(kx, ky, background_permittivity)

    r_B = 1.5
    theta_B = np.linspace(0, 2* pi, 361)[0:-1]

    pts = [(r_B * cos(theta_B[i]), r_B * sin(theta_B[i])) for i in range(len(theta_B))]


    u_scat_vals = [u_scat(dg.mesh(*pt)) for pt in pts]
    grad_u_scat_vals = [ (grad(u_scat)(dg.mesh(*pt))[0] * pt[0] + grad(u_scat)(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]


    for j in range(4):
        p_kx = 2 * pi * cos(direction_angles[j])
        p_ky = 2 * pi * sin(direction_angles[j])

        psi2 = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))

        uB_scat = dg.solve(p_kx, p_ky, background_permittivity)


        # dg.save(u_scat, "scatter", "data/scatter.mat")
        grad_uB_scat = grad(uB_scat)


        uB_scat_vals = [uB_scat(dg.mesh(*pt)) for pt in pts]
        grad_uB_scat_vals = [ (grad_uB_scat(dg.mesh(*pt))[0] * pt[0] + grad_uB_scat(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]


        # define domain B as a disk of radius 1.5
        # generate Cauchy data

        psi_test1 = CF((exp(1j * kx * x) * exp(1j * ky * y)))
        grad_psi_test_X1 = psi_test1.Diff(x)
        grad_psi_test_Y1 = psi_test1.Diff(y)

        psi_vals1 = [psi_test1(dg.mesh(*pt)) for pt in pts]
        grad_psi_vals1 = [ (grad_psi_test_X1(dg.mesh(*pt)) * pt[0] + grad_psi_test_Y1(dg.mesh(*pt)) * pt[1]) / r_B  for pt in pts ]



        psi_test2 = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))
        grad_psi_test_X2 = psi_test2.Diff(x)
        grad_psi_test_Y2 = psi_test2.Diff(y)

        psi_vals2 = [psi_test2(dg.mesh(*pt)) for pt in pts]
        grad_psi_vals2 = [ (grad_psi_test_X2(dg.mesh(*pt)) * pt[0] + grad_psi_test_Y2(dg.mesh(*pt)) * pt[1]) / r_B  for pt in pts ]


        # s = sum( [grad_psi_vals[i] * uB_scat_vals[i] - grad_uB_scat_vals[i] * psi_vals[i] for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)

        # print(s)

        # s = sum( [(grad_uB_scat_vals[i] + grad_psi_vals2[i]) * (u_scat_vals[i]+ psi_vals1[i]) - (grad_u_scat_vals[i]+ grad_psi_vals1[i]) * (uB_scat_vals[i] + psi_vals2[i]) for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)




        true_val = Integrate( (permittivity - background_permittivity) * (uB_scat + psi2) *  (u_scat + psi1), dg.mesh)
        cur_val =  Integrate( (permittivity - background_permittivity) * (uB_scat + psi2) *  (uB_scat0 + psi1), dg.mesh)

        print(i,j, true_val, cur_val, true_val - cur_val)

        tmp = dg.fes.TrialFunction()
        tmp2 = dg.fes.TestFunction()

        linear_form = LinearForm(dg.fes)

        linear_form += tmp2 * (uB_scat + psi2) *  (u_scat + psi1) * (IfPos((x)**2 + (y)**2 - (0.5) **2,
                 0,
                 1))  * dx

        linear_form.Assemble()

        A.NumPy()[l, :] = linear_form.vec.FV().NumPy()
        b[l] = true_val
        l += 1

Solving took 4.9415055809076875 seconds
Solving took 4.728149355156347 seconds
Solving took 4.69840709795244 seconds
0 0 (-0.014520303344383027-0.019914493462974556j) (0.00737502119285764-9.269169733247973e-10j) (-0.02189532453724067-0.019914492536057583j)
Solving took 4.695672995178029 seconds
0 1 (0.01464560216727575+0.03404207568059099j) (0.06307755846370189-6.827303193073669e-10j) (-0.04843195629642614+0.03404207636332131j)


In [34]:
from numpy.linalg import lstsq

v = lstsq(A.NumPy(), b.NumPy())[0]

/tmp/ipykernel_2136509/3633902184.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  v = lstsq(A.NumPy(), b.NumPy())[0]


array([-7.45931095e-17-6.93889390e-18j, -1.42247325e-16+0.00000000e+00j,
       -2.77555756e-17+4.16333634e-17j,  1.73472348e-17-1.11022302e-16j,
        1.49186219e-16+2.98372438e-16j, -8.67361738e-18+0.00000000e+00j,
        1.38777878e-17-6.93889390e-17j,  2.74780199e-15+1.73472348e-15j,
       -5.55111512e-16-3.19189120e-16j,  2.94902991e-17+2.77555756e-17j,
        0.00000000e+00+1.73472348e-17j, -5.37764278e-17-1.31838984e-16j,
        3.50414142e-16+7.21644966e-16j, -1.94289029e-16-5.55111512e-17j,
       -3.64291930e-17-9.71445147e-17j, -8.15320034e-17-7.63278329e-17j])

In [54]:
vec_v = Vector(v)

u_scat_copy = u_scat

u_scat_copy.vec.data = v

scene = Draw(u_scat_copy, dg.mesh)
clear_output() # redraw the scene with a new height
scene.Draw(height="3vh")


WebGuiWidget(layout=Layout(height='3vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'spe…

In [53]:

u_scat_copy.vec.data = v

(5.227209923938116e-05+4.486618639898063e-05j)

In [32]:
b.NumPy()

array([-0.02187535-0.01982756j,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ])

In [10]:
r_B = 1.5
theta_B = np.linspace(0, 2* pi, 1025)[0:-1]

pts = [(r_B * cos(theta_B[i]), r_B * sin(theta_B[i])) for i in range(len(theta_B))]

uB_scat_vals = [uB_scat(dg.mesh(*pt)) for pt in pts]
grad_uB_scat_vals = [ (grad_uB_scat(dg.mesh(*pt))[0] * pt[0] + grad_uB_scat(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]


u_scat_vals = [u_scat(dg.mesh(*pt)) for pt in pts]
grad_u_scat_vals = [ (grad(u_scat)(dg.mesh(*pt))[0] * pt[0] + grad(u_scat)(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]

# define the test function
p_kx =  kx
p_ky =  ky
psi_test = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))
grad_psi_test_X = psi_test.Diff(x)
grad_psi_test_Y = psi_test.Diff(y)

psi_vals = [psi_test(dg.mesh(*pt)) for pt in pts]
grad_psi_vals = [ (grad_psi_test_X(dg.mesh(*pt)) * pt[0] + grad_psi_test_Y(dg.mesh(*pt)) * pt[1]) / r_B  for pt in pts ]

# s = sum( [grad_psi_vals[i] * uB_scat_vals[i] - grad_uB_scat_vals[i] * psi_vals[i] for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)

# print(s)

s = sum( [(grad_psi_vals[i] + grad_uB_scat_vals[i]) * (u_scat_vals[i]-uB_scat_vals[i]) - (grad_u_scat_vals[i]- grad_uB_scat_vals[i]) * (psi_vals[i]+uB_scat_vals[i]) for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)
print(s)



Integrate( -(permittivity - background_permittivity) * (uB_scat) *  (uB_scat), dg.mesh)

# r_B = 1.5
# theta_B = np.linspace(0, 2* pi, 1025)[0:-1]

# pts = [(r_B * cos(theta_B[i]), r_B * sin(theta_B[i])) for i in range(len(theta_B))]

# uB_scat_vals = [uB_scat(dg.mesh(*pt)) for pt in pts]
# grad_uB_scat_vals = [ (grad_uB_scat(dg.mesh(*pt))[0] * pt[0] + grad_uB_scat(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]


# u_scat_vals = [u_scat(dg.mesh(*pt)) for pt in pts]
# grad_u_scat_vals = [ (grad(u_scat)(dg.mesh(*pt))[0] * pt[0] + grad(u_scat)(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]

# # define the test function
# p_kx =  0 * pi
# p_ky =  8 * pi 
# psi_test = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))
# grad_psi_test_X = psi_test.Diff(x)
# grad_psi_test_Y = psi_test.Diff(y)

# psi_vals = [psi_test(dg.mesh(*pt)) for pt in pts]
# grad_psi_vals = [ (grad_psi_test_X(dg.mesh(*pt)) * pt[0] + grad_psi_test_Y(dg.mesh(*pt)) * pt[1]) / r_B  for pt in pts ]

# s = sum( [grad_psi_vals[i] * uB_scat_vals[i] - grad_uB_scat_vals[i] * psi_vals[i] for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)

# print(s)

# s = sum( [grad_psi_vals[i] * u_scat_vals[i] - grad_u_scat_vals[i] * psi_vals[i] for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)
# print(s)

(-0.00534794670099449-0.016631727305240263j)


(-0.0049966245780328974+0.006142166691137205j)

In [18]:
scene = Draw(u_scat-uB_scat, dg.mesh)
clear_output() # redraw the scene with a new height
scene.Draw(height="3vh")

WebGuiWidget(layout=Layout(height='3vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.240…